In [1]:
import numpy as np
import pandas as pd
import regex as re
"""pip install contractions"""
import contractions
import string


In [2]:
data = pd.read_csv("discharge.csv",nrows=2000)

In [3]:
#dropping unnecessary columns
data = data.drop(columns= ['subject_id','hadm_id','note_type','charttime','storetime','note_id','note_seq'])  

#removing new line characters from text
data['text'] = data['text'].replace('\n','', regex=True) 

#expanding contractions
data['text'] = data['text'].apply(lambda txt:contractions.fix(txt)) 

#converting text to lower case
data['text'] = data['text'].apply(lambda txt: txt.lower()) 

#removing unnecessary text
data['text'] = data['text'].replace(' name: ___ unit no: ___ admission date: ___ discharge date: ___ date of birth: ___ ','',regex=True) 
data['text'] = data['text'].replace('followup instructions:___','',regex=True)  

#removing numbers from text
data['text'] = data['text'].str.replace("(\s\d+)+\s", " ", regex=True)  
data['text'] = data['text'].str.replace(r'\d+', '', regex=True)

#removing special characters from text
data['text'] = data['text'].str.replace("[^A-Za-z0-9]+", " ", regex=True) 

#removing extra spaces from text
data['text'] = data['text'].str.replace("\s+", " ", regex=True) 

#removing special characters "_ ","="" from text
data['text'] = data['text'].apply(lambda txt: re.sub('[%s]' % re.escape('_='),'',txt))  

In [4]:
from nltk.corpus import stopwords
#get the list of stopwords from NLTK library
mystpwrd = stopwords.words('english') 
mystpwrd.remove('not')

#custom stopwords
custom_stopwords = {"name", "unit", "no", "admission", "date", "discharge", "of", "birth", "sex", "service", "or", "and", "known", "with", "this", "is", "attending"}
mystpwrd = mystpwrd + list(custom_stopwords)

#removing stopwords from text
data['text'] = data['text'].apply(lambda txt: " ".join([word for word in str(txt).split() if word not in mystpwrd]))


https://www.analyticsvidhya.com/blog/2021/06/must-known-techniques-for-text-preprocessing-in-nlp/

In [5]:
#Lemmatization of text
#pip install nltk.stem
"""import nltk
nltk.download('wordnet')"""
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
data["text"] = data["text"].apply(lambda text: lemmatize_words(text))

In [6]:
data.to_csv('Preprocessing_discharge.csv', index=False) 